In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Input, Layer, concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import xgboost as xgb
import imageio
from os import listdir
from os.path import isfile, join
from tensorflow import keras
import shutil
import os
import PIL.Image
import glob

In [ ]:
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv',index_col=[0])

In [ ]:
test_images_folder = "../input/petfinder-pawpularity-score/test"

In [ ]:
def listing_function(y, x_train_1, images_folder):
    expectations_from_model_1 = []
    imagees = []
    answers = []
    train_images_folder_ready = glob.glob(images_folder + '/*.jpg')
    train_images_folder_ready.sort(reverse=False)

    for imagess in train_images_folder_ready:
            m = (os.path.basename(imagess))
            verify = (m)
            verify = os.path.splitext(verify)
            verify = verify[0]
            for i in x_train_1.index:
                    if verify == i:
                            x_lt = tf.io.read_file(imagess)
                            x_lt = tf.image.decode_jpeg(x_lt, channels=3)
                            x_lt = tf.cast(x_lt, tf.float32) / 255.0
                            x_lt = tf.image.resize(x_lt, (200,200))
                            imagees.append(x_lt)
                            
                            x_vals_final = x_train_1.loc[i]
                            x_vals_final = np.array(x_vals_final)
                            expectations_from_model_1.append(x_vals_final)
                            
                            if len(y)>=1:
                                y_final = y[i]
                                y_final = np.array(y_final)
                                answers.append([y_final])
                            else:
                                answers = []

    return expectations_from_model_1, imagees, answers

In [ ]:
exp_features, test_img_final, _ = listing_function([], test_csv, test_images_folder)

In [ ]:
array_of_test_img_final = np.array(test_img_final)
array_of_exp_features = np.array(exp_features)

In [ ]:
import pickle

In [ ]:
Pkl_Filename = "../input/kksindia/Pickle_xg_Model.pkl"  
with open(Pkl_Filename, 'rb') as file:  
    xgb1 = pickle.load(file)

In [ ]:
new_model = tf.keras.models.load_model('../input/kksindia/super_beginner_trained_model.pkl')

In [ ]:
final_features =new_model.predict(array_of_test_img_final)

In [ ]:
final_features = new_model.predict(array_of_test_img_final)

In [ ]:
final_features_1 = final_features*2
final_features_12 = final_features_1.astype(int)
final_features_12

In [ ]:
result = xgb1.predict(final_features)

In [ ]:
finale = pd.DataFrame()
finale['Id'] = test_csv.index
finale['Pawpularity'] = result
finale.to_csv('submission.csv',index=False)